In [144]:
import numpy as np_ # Original numpy 
import scipy as sp
from enum import Enum


import random

# Plot
from IPython.display import display, Latex
import matplotlib.pyplot as plt

from matplotlib.ticker import MultipleLocator

# PennyLane
import pennylane as qml
from pennylane import numpy as np

# qnode device
dev = qml.device('default.mixed', wires = 1)

# Hamiltonian setup 
coeffs = [-0.5]
obs = [qml.PauliZ(0)]

hamiltonian = qml.Hamiltonian(coeffs, obs)  # Define Hamiltonian

In [145]:
class Data_Index(Enum):
    PHI = 0
    THETA_X = 1
    PHI_Z = 2
    

In [146]:
Data_Index.THETA_X.value

1

In [147]:
# Global paras 
Tau_global = 5e-2   # Dephase tau
Paras_global, Phi_global  = np.zeros(2), 0

def Dephase_factor(tau):
    """ Take tau and return gamma based on the following relation."""

    Calculate_Gamma = 1 - np.exp(-2 * tau)
    
    return Calculate_Gamma    # Return gamma

In [148]:
test = np.array([
    [0.1, np.pi/2, np.pi/2],
    [0.2, np.pi/2, np.pi/2] 
])

In [149]:
@qml.qnode(dev)
def circuit(paras):
    
    # Set dephase factor 

    # theta_x, phi_z = paras[Data_Index.THETA_X.value], paras[Data_Index.PHI_Z.value]
    theta_x = paras[Data_Index.THETA_X.value]

    qml.RX(np.pi/2, wires = 0)
    # qml.Hadamard(wires=0)

    qml.ApproxTimeEvolution(hamiltonian, paras[Data_Index.PHI.value], 1)

    qml.RZ(np.pi/2, wires = 0)  # phi_z
    
    qml.RX(theta_x, wires = 0)  # theta_x
    
    return qml.density_matrix(wires = 0)



In [150]:
Data_set = np.array([
    [0.1, np.pi/2, np.pi/2],
    [0.2, np.pi/2, np.pi/2] 
])

In [151]:
Data_set[0][0]

tensor(0.1, requires_grad=True)

In [152]:
print(qml.qinfo.classical_fisher(circuit)(Data_set[0]))

[[1.00000000e+00 3.06797807e-17 0.00000000e+00]
 [3.06797807e-17 1.71162092e-33 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]]


In [153]:
def Cost_function(paras):
    """ Calculate Classical-Fisher-Information for qnode(=Post_selection_Dephase).
    
    Args:
        paras (Numpy array): [theta_init, tau_1, tau_2, tau_d1, tau_d2, tau_d3]

    Returns:
        _type_: CFI with minus(-) sign.
    """
    
    global Paras_global, Phi_global
    Paras_global, phi = paras, np.array([Phi_global])
          
    CFI = qml.qinfo.classical_fisher(circuit)(phi[0])
    
    return -CFI

In [154]:
test = np.array([1,2,3])

test[:]

tensor([1, 2, 3], requires_grad=True)